In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [3]:
class CharacterTokenizer:
    def __init__(self):
        self.char_to_index = {chr(i): i for i in range(128)}

    def encode(self, text):
        return [self.char_to_index.get(c, 0) for c in text]

In [4]:
class CharacterDataset(Dataset):
    def __init__(self, texts, labels):
        self.tokenizer = CharacterTokenizer()
        self.texts = [self.tokenizer.encode(text) for text in texts]
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

In [5]:
class BiLSTM(nn.Module):
    def __init__(self, num_chars, num_labels, embedding_dim=64, hidden_dim=128):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(num_chars, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        self.to(device)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out)
        return out

In [6]:
def train_model(model, data_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1, outputs.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(data_loader)}')

In [7]:
def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, -1)
            all_preds.extend(predicted.cpu().view(-1).numpy())
            all_labels.extend(labels.cpu().view(-1).numpy())
    print(classification_report(all_labels, all_preds))

In [8]:
# Example data
texts = ["hello", "world"]  # example text data
labels = [[1, 0, 0, 0, 2], [2, 0, 0, 0, 1]]  # Corresponding labels for each character

# Setup dataset and DataLoader
dataset = CharacterDataset(texts, labels)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Initialize model, optimizer, and loss function
num_chars = 128  # ASCII range
num_labels = 3   # Number of different labels
model = BiLSTM(num_chars, num_labels)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignoring PAD label

# Train and evaluate the model
train_model(model, data_loader, optimizer, criterion, epochs=10)
evaluate_model(model, data_loader)